In [1]:
import pandas as pd
test = pd.read_csv('data\history\BCHUSDT-15m-data.csv')

In [1]:
from environment.env import TradingEnv

In [2]:
env = TradingEnv('data/history', commission=0.01, time_cost=0.05, steps=800, start_date_index=0, augment=0.05)

In [5]:
env.action_space.sample()

array([0.11358929, 0.9228822 , 0.9576744 , 0.4205666 , 0.18355837,
       0.07581317, 0.7203717 , 0.2727475 , 0.03127049, 0.56743574,
       0.92260927], dtype=float32)

In [3]:
env.observation_space.sample()['observation'].shape

(4, 30, 11)

In [4]:
obs = env.reset()
obs['observation'].shape

(4, 30, 10)

In [1]:
import numpy as np

In [8]:
obs = np.random.random([4, 60, 8])
weight = np.random.randn(9)

In [7]:
weight[1:, np.newaxis, np.newaxis].shape

(8, 1, 1)

In [10]:
obs.shape

(4, 60, 8)